In [67]:
import numpy as np
import copy

In [85]:
class Process:
    
    def set_name(self,name):
        self.name = name
        
    def set_memory_MB(self,memory_MB):
        self.memory_MB = memory_MB
        
    def set_time_s(self,time_s):
        self.time_s = time_s
        
    def set_input_size_MB(self,input_MB):
        self.input_size_MB = input_MB
        
    def set_output_size_MB(self,output_MB):
        self.output_size_MB = output_MB
    
    def set_threads(self,min_threads,max_threads=None):
        self.min_threads = min_threads
        if max_threads:
            self.max_threads = max_threads
        else:
            self.max_threads = min_threads
    
    def set_pass_fraction(self,pass_fraction):
        self.pass_fraction = pass_fraction
        
    def set_failure_rate(self,failure_rate):
        self.failure_rate = failure_rate
    
    def __init__(self,
                 memory_MB=0,time_s=0,
                 input_MB=0,output_MB=0,
                 name="",
                 min_threads=1,max_threads=None,
                 pass_fraction=1.0,
                 failure_rate=0.0):

        self.set_name(name)
        
        self.set_memory_MB(memory_MB)
        self.set_time_s(time_s)
        
        self.set_input_size_MB(input_MB)
        self.set_output_size_MB(output_MB)
        
        self.set_threads(min_threads,max_threads)

        self.set_pass_fraction(pass_fraction)
        self.set_failure_rate(failure_rate)

    def __str__(self):
        s = "Process '%s':"%self.name
        s = s+"\n\tMemory (MB): %f"%self.memory_MB
        s = s+"\n\tTime per event (s): %f"%self.time_s
        s = s+"\n\tInput/Output size per event (MB): %f / %f"%(self.input_size_MB,self.output_size_MB)
        s = s+"\n\tThreads used (min,max): (%d,%d)"%(self.min_threads,self.max_threads)
        s = s+"\n\tEvent pass fraction: %f"%self.pass_fraction
        s = s+"\n\tFailure rate: %f"%self.failure_rate
        return s
        
def ChainProcesses(processes,name=""):
    
    if len(processes)==0:
        print("Process list length 0. Return empty Process.")
        return Process()
    
    max_memory = np.max([p.memory_MB for p in processes])

    time_s = processes[0].time_s
    for i in range(1,len(processes)):
        time_s += np.prod([p.pass_fraction for p in processes[0:i]])*processes[i].time_s
    
    input_size_MB = processes[0].input_size_MB
    max_input_size = np.max([p.input_size_MB for p in processes])
    if input_size_MB < max_input_size:
        print("WARNING! First process input size (%f) is less than max input size (%f).\nSetting max to %f MB"%
              (input_size_MB,max_input_size,max_input_size))
        input_size_MB=max_input_size
            
    min_threads = np.max([p.min_threads for p in processes])
    max_threads = np.max([p.max_threads for p in processes])
    
    pass_fraction = np.prod([p.pass_fraction for p in processes])
    failure_rate = 1. - np.prod([(1.- p.failure_rate) for p in processes])
    
    output_size_MB = processes[-1].output_size_MB*pass_fraction

    return Process(memory_MB=max_memory,time_s=time_s,
                   name=name,
                   input_MB=input_size_MB,output_MB=output_size_MB,
                   min_threads=min_threads,max_threads=max_threads,
                   pass_fraction=pass_fraction,failure_rate=failure_rate)

In [99]:
class DataIOInterface:
    
    def set_name(self,name):
        self.name = name

    def set_rate_MBps(self,rate_MBps):
        self.rate_MBps = rate_MBps
    
    def __init__(self,
                 name="",
                 rate_MBps=np.inf):
        
        self.set_name(name)
        self.set_rate_MBps(rate_MBps)
        

In [101]:
class Job:
    
    DEFAULT_INTERFACE = DataIOInterface(name="Default",rate_MBps=np.inf)
    
    def set_process(self,process,recompute=True):
        if type(process) is list:
            process = ChainProcesses(process)
        self.process=process
        if recompute: self.recompute()
        
    def set_n_events(self,n_events,recompute=True):
        self.n_events = n_events
        if recompute: self.recompute()
        
    def recompute(self):
        self.memory_MB = self.process.memory_MB
        self.cpu_time_s = self.process.time_s * self.n_events

        self.input_size_MB = self.process.input_size_MB * self.n_events
        self.input_time_s = self.input_size_MB / self.input_interface.rate_MBps
        
        self.output_size_MB = self.process.output_size_MB * self.n_events * self.process.pass_fraction
        self.output_time_s = self.output_size_MB / self.output_interface.rate_MBps
        
        self.io_time_s = self.input_time_s + self.output_time_s
        self.total_time_s = self.cpu_time_s + self.io_time_s        
        
        self.pass_fraction = self.process.pass_fraction
        self.failure_rate = 1. - np.power((1.-self.process.failure_rate),self.n_events)
    
    def set_input_interface(self,input_interface,recompute=True):
        self.input_interface=input_interface
        if recompute: self.recompute()
    
    def set_output_interface(self,output_interface,recompute=True):
        self.output_interface=output_interface
        if recompute: self.recompute()

    def __init__(self,
                 process,
                 input_interface=DEFAULT_INTERFACE,
                 output_interface=DEFAULT_INTERFACE,
                 n_events=100):
        
        self.set_process(process,recompute=False)
        self.set_n_events(n_events,recompute=False)

        self.set_input_interface(input_interface,recompute=False)
        self.set_output_interface(output_interface,recompute=False)
        
        self.recompute()
        
    def __str__(self):
        s = "Process: '%s':"%self.process.name
        s = s+"\nN_Events: %d"%self.n_events
        s = s+"\nMemory (MB): %f"%(self.memory_MB)
        s = s+"\nTime (hr): %f"%(self.total_time_s/3600.)
        s = s+"\n\tCPU Time (hr): %f"%(self.cpu_time_s/3600.)
        s = s+"\n\tIO Time (hr): %f"%(self.io_time_s/3600.)
        s = s+"\nEvent pass fraction: %f"%self.pass_fraction
        s = s+"\nInput/Output size (GB): %f / %f"%(self.input_size_MB/1000.,self.output_size_MB/1000.)
        s = s+"\nFailure rate: %f"%self.failure_rate
        return s

In [ ]:
class Sample:
    
    def __init__(self,
                 job,
                 n_events=None,
                 n_jobs=None):
        

In [20]:
p = Process(name="unnamed")

In [21]:
print(p)

Process 'unnamed'
	Memory (MB): 0.000000
	Time per event (s): 0.000000
	Input/Output size per event (MB): 0.000000 / 0.000000
	Threads used (min,max): (1,1)
	Event pass fraction: 1.000000
	Failure rate: 0.000000


In [72]:
icarus_data_reco1 = Process(name="ICARUS Data Stage0",
                           memory_MB=4000,
                           time_s=120,
                           input_MB=170,
                           output_MB=50)

In [73]:
icarus_data_reco2 = Process(name="ICARUS Data Stage1",
                           memory_MB=4000,
                           time_s=100,
                           input_MB=50,
                           output_MB=45)

In [74]:
print(icarus_data_reco1)
print(icarus_data_reco2)

Process 'ICARUS Data Stage0':
	Memory (MB): 4000.000000
	Time per event (s): 120.000000
	Input/Output size per event (MB): 170.000000 / 50.000000
	Threads used (min,max): (1,1)
	Event pass fraction: 1.000000
	Failure rate: 0.000000
Process 'ICARUS Data Stage1':
	Memory (MB): 4000.000000
	Time per event (s): 100.000000
	Input/Output size per event (MB): 50.000000 / 45.000000
	Threads used (min,max): (1,1)
	Event pass fraction: 1.000000
	Failure rate: 0.000000


In [75]:
icarus_data_reco = ChainProcesses([icarus_data_reco1,icarus_data_reco2],"ICARUS Data Reco")

In [76]:
print(icarus_data_reco)

Process 'ICARUS Data Reco':
	Memory (MB): 4000.000000
	Time per event (s): 220.000000
	Input/Output size per event (MB): 170.000000 / 45.000000
	Threads used (min,max): (1,1)
	Event pass fraction: 1.000000
	Failure rate: 0.000000


In [57]:
icarus_data_reco1_job = Job(icarus_data_reco1,50)

In [58]:
print(icarus_data_reco1_job)

Process: 'ICARUS Data Stage0':
N_Events: 50
Memory (MB): 4000.000000
Time (hr): 1.666667
Event pass fraction: 1.000000
Input/Output size (GB): 8.500000 / 2.500000
Failure rate: 0.000000


In [59]:
icarus_data_reco_job = Job([icarus_data_reco1,icarus_data_reco2],50)

In [60]:
print(icarus_data_reco_job)

Process: '':
N_Events: 50
Memory (MB): 4000.000000
Time (hr): 3.055556
Event pass fraction: 1.000000
Input/Output size (GB): 8.500000 / 2.250000
Failure rate: 0.000000


In [68]:
icarus_data_reco1_notrig = copy.copy(icarus_data_reco1)
icarus_data_reco1_notrig.set_pass_fraction(0.05)

In [69]:
print(icarus_data_reco1)

Process 'ICARUS Data Stage0':
	Memory (MB): 4000.000000
	Time per event (s): 120.000000
	Input/Output size per event (MB): 170.000000 / 50.000000
	Threads used (min,max): (1,1)
	Event pass fraction: 1.000000
	Failure rate: 0.000000


In [70]:
print(icarus_data_reco1_notrig)

Process 'ICARUS Data Stage0':
	Memory (MB): 4000.000000
	Time per event (s): 120.000000
	Input/Output size per event (MB): 170.000000 / 50.000000
	Threads used (min,max): (1,1)
	Event pass fraction: 0.050000
	Failure rate: 0.000000


In [80]:
icarus_data_reco_notrig = ChainProcesses([icarus_data_reco1_notrig,icarus_data_reco2],"ICARUS Data Reco NoTrig")

In [81]:
print(icarus_data_reco_notrig)

Process 'ICARUS Data Reco NoTrig':
	Memory (MB): 4000.000000
	Time per event (s): 125.000000
	Input/Output size per event (MB): 170.000000 / 2.250000
	Threads used (min,max): (1,1)
	Event pass fraction: 0.050000
	Failure rate: 0.000000


In [90]:
icarus_data_pmtFilter = Process(name="ICARUS Data PMTFilter",
                                memory_MB=2000,
                                time_s=2,
                                input_MB=170,
                                output_MB=50*0.05,
                                pass_fraction=0.05)

In [86]:
icarus_data_reco_notrig = ChainProcesses([icarus_data_pmtFilter,icarus_data_reco1,icarus_data_reco2],"ICARUS Data Reco NoTrig")

WARNING! First process input size (8.500000) is less than max input size (170.000000).
Setting max to 170.000000 MB


In [87]:
print(icarus_data_reco_notrig)

Process 'ICARUS Data Reco NoTrig':
	Memory (MB): 4000.000000
	Time per event (s): 13.000000
	Input/Output size per event (MB): 170.000000 / 2.250000
	Threads used (min,max): (1,1)
	Event pass fraction: 0.050000
	Failure rate: 0.000000


In [103]:
icarus_data_reco_job_pmtfilter = Job([icarus_data_pmtFilter,icarus_data_reco1,icarus_data_reco2],n_events=50)

In [105]:
tape_io_interface = DataIOInterface(name="FNAL Enstore",rate_MBps=300)

In [106]:
icarus_data_reco_job_pmtfilter = Job(process=[icarus_data_pmtFilter,icarus_data_reco1,icarus_data_reco2],
                                     n_events=50,
                                     input_interface=tape_io_interface,
                                     output_interface=tape_io_interface)

In [107]:
print(icarus_data_reco_job_pmtfilter)

Process: '':
N_Events: 50
Memory (MB): 4000.000000
Time (hr): 0.188431
	CPU Time (hr): 0.180556
	IO Time (hr): 0.007876
Event pass fraction: 0.050000
Input/Output size (GB): 8.500000 / 0.005625
Failure rate: 0.000000


In [96]:
1./np.inf

0.0